# Exploring neural networks

## Problem 1

Given a dataset where each sample contains all 0s and randomly a 1 somewhere to replace the 0s.
Create a neural network that can distinguish between record containing all 0s and those that contain 0s and a 1.

In [274]:
import numpy as np
import random

In [275]:
a = np.zeros(10)
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [276]:
def generate_dataset(size = 1000):

    inp = []
    labels =[]

    for i in range(size):
        inp.append(np.zeros(11))
        labels.append(0)
        rnd_choice = random.choice([0,1])
        inp[i][random.choice(range(10))] = rnd_choice
        labels[i] = rnd_choice
   
    return (np.array(inp), labels)
    

In [277]:
ds = generate_dataset()


In [278]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(11, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=11, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [279]:
def torch_train_test_data(split=800):
    
    x_train = torch.tensor(ds[0][0:split], dtype=torch.float32)
    y_train = torch.tensor(ds[1][0:split], dtype=torch.long)
    
    train_data = data.TensorDataset(x_train, y_train)

    x_test = torch.tensor(ds[0][split:], dtype=torch.float32)
    y_test = torch.tensor(ds[1][split:], dtype=torch.long)

    test_data = data.TensorDataset(x_test, y_test)
       
    return (x_train, y_train, x_test, y_test, train_data, test_data)


x_train, y_train, x_test, y_test, train_data, test_data = torch_train_test_data()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4,
                                         shuffle=False, num_workers=0) 
    
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4,
                                         shuffle=False, num_workers=0)


In [280]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [284]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
#         print(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.001
[1,   200] loss: 0.001
[2,   100] loss: 0.001
[2,   200] loss: 0.001
[3,   100] loss: 0.001
[3,   200] loss: 0.001
[4,   100] loss: 0.001
[4,   200] loss: 0.001
[5,   100] loss: 0.001
[5,   200] loss: 0.000
[6,   100] loss: 0.000
[6,   200] loss: 0.000
[7,   100] loss: 0.000
[7,   200] loss: 0.000
[8,   100] loss: 0.000
[8,   200] loss: 0.000
[9,   100] loss: 0.000
[9,   200] loss: 0.000
[10,   100] loss: 0.000
[10,   200] loss: 0.000
Finished Training


In [282]:
outputs = net(torch.tensor([0,1,0,0,0,0,0,0,0,0,0], dtype=torch.float32))

predicted = torch.max(outputs, 0)

predicted[1].data.cpu().numpy()

array(1)

In [283]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        input_data, labels = data
        # Note input data is batched, so output is batched
        outputs = net(input_data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (x_test.size()[0],
    100 * correct / total))

Accuracy of the network on the 200 test images: 100 %
